###데이터 다운로드

In [1]:
import requests
res = requests.get('https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip')
with open('sentiment labelled sentences.zip', 'wb') as f:
    f.write(res.content)

###데이터 압축 해제

In [3]:
from zipfile import ZipFile
z = ZipFile('sentiment labelled sentences.zip')
data = z.open('sentiment labelled sentences/amazon_cells_labelled.txt')

###파일오픈

In [4]:
import pandas as pd

df = pd.read_csv(data, sep="\t", header=None)

In [5]:
df.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [7]:
df.shape

(1000, 2)

###단어 문서 행렬만들기

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer(max_features=1000, stop_words='english')

In [15]:
tdm = cv.fit_transform(df[0])
df[0]

0      So there is no way for me to plug it in here i...
1                            Good case, Excellent value.
2                                 Great for the jawbone.
3      Tied to charger for conversations lasting more...
4                                      The mic is great.
                             ...                        
995    The screen does get smudged easily because it ...
996    What a piece of junk.. I lose more calls on th...
997                         Item Does Not Match Picture.
998    The only thing that disappoint me is the infra...
999    You can not answer calls with the unit, never ...
Name: 0, Length: 1000, dtype: object

###데이터 분할

In [16]:
x = tdm
y = df[1]

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [19]:
x_train

<800x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 3220 stored elements in Compressed Sparse Row format>

In [20]:
x_test

<200x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [21]:
y_train

29     0
535    0
695    0
557    1
836    0
      ..
106    0
270    0
860    0
435    0
102    1
Name: 1, Length: 800, dtype: int64

In [22]:
y_test

521    1
737    1
740    1
660    0
411    1
      ..
408    1
332    1
208    1
613    1
78     1
Name: 1, Length: 200, dtype: int64

In [23]:
import joblib

joblib.dump((x_train, x_test, y_train, y_test), 'review.pkl')

['review.pkl']

In [24]:
import tensorflow as tf

In [31]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, input_shape=(1000,), activation='sigmoid'))

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 1001      
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [35]:
x_train = x_train.toarray()

model.fit(x_train, y_train, epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6913 - accuracy: 0.5387
Epoch 2/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6817 - accuracy: 0.6012
Epoch 3/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6727 - accuracy: 0.6725
Epoch 4/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6638 - accuracy: 0.7212
Epoch 5/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6552 - accuracy: 0.7700
Epoch 6/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6467 - accuracy: 0.8025
Epoch 7/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6384 - accuracy: 0.8263
Epoch 8/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6303 - accuracy: 0.8450
Epoch 9/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6223 - accuracy: 0.8625
Epoch 10/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6146 - accuracy: 0.8750


In [37]:
x_test = x_test.toarray()

model.evaluate(x_test, y_test)

7/7 [==============================] - 0s 2ms/step - loss: 0.6404 - accuracy: 0.7450


[0.6403993368148804, 0.7450000047683716]

In [38]:
weight, bias = model.trainable_weights

In [39]:
import pandas as pd
word_weight = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '가중치': weight.numpy().flat
})

In [40]:
word_weight.sort_values('가중치').head()

,단어,가중치
220,disappointed,-0.188411
938,waste,-0.180068
140,calls,-0.176760
319,having,-0.169942
982,work,-0.169855


In [41]:
word_weight.sort_values('가중치', ascending=False).head()

,단어,가중치
310,great,0.224744
395,love,0.186839
294,free,0.176463
316,happy,0.169565
255,excellent,0.162868
